# Model Evaluation
***

In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris 
from sklearn.linear_model import LogisticRegression

In [2]:
iris = load_iris()
logreg = LogisticRegression(max_iter=500)
scores = cross_val_score(
    logreg,
    iris.data,
    iris.target,
)
print(scores)
print(f"Score da validação cruzada: {scores.mean():.3f} +/- {scores.std():.3f}")

[0.96666667 1.         0.93333333 0.96666667 1.        ]
Score da validação cruzada: 0.973 +/- 0.025


Stratified k-fold e outras estratégias

In [3]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits = 10, shuffle = True, random_state = 123)
scores = cross_val_score(
    logreg,
    iris.data,
    iris.target,
    cv = kfold
)
print(scores)
print(f"Score da validação cruzada: {scores.mean():.3f} +/- {scores.std():.3f}")

[1.         0.93333333 0.93333333 1.         1.         0.93333333
 0.93333333 1.         1.         0.86666667]
Score da validação cruzada: 0.960 +/- 0.044


## Avaliar a performance do modelo depois da seleção do modelo
***

Solução usar nested cross-validation para evitar avaliação enviesada.

In [4]:
# Load libraries
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV, cross_val_score

# Load data
iris = datasets.load_iris()
features = iris.data
target = iris.target

# Create logistic regression
logistic = linear_model.LogisticRegression(max_iter=500)

# Create range of 20 candidate values para C
C = np.logspace(0, 4, 20)

# Create hyperparameter options
hyperparameters = dict(C = C)

# Create grid search
grid_search = GridSearchCV(
    logistic,
    hyperparameters,
    cv=5, 
    n_jobs=-1,
    verbose = 0,
)

# Conduct nested cross-validation and output the average score
cv_results = cross_val_score(
    grid_search, 
    features, 
    target,
)
scores = cv_results
print(f"A média da acurácia no cross-validation é: {scores.mean():.3f} +/- {scores.std():.3f}")

A média da acurácia no cross-validation é: 0.967 +/- 0.030


Discussão sobre nested cross-validation.

Nested cross-validation durante a seleção do modelo é um conceito muito bem complicado para muitas pessoas entenderem da primeira vez. Lembre-se que em k-fold cross-validation, nós treinamos o modelo em $k - 1$ folds dos dados, usamos este modelo para fazer predições no fold remanescente e avaliamos o nosso melhor modelo em como as predições estão boas comparadas com os valores reais. Nós então repetimos esse processo $k$ vezes.

Na procura pelo melhor modelo, podemos usar **GridSearchCV** ou **RandomizedSearchCV**, nós usamos cross-validation para avaliar que hiperparâmetros, produzem os melhores modelos. Contudo, surge um problema, já que usamos os dados para selecionar os melhores hiperparâmetros, não podemos usar os mesmos dados para avaliar a performance do modelo. A solução? Envelopar a cross-validation usada na procura do modelo em outra cross-validation. Nesta cross-validation aninhada(nested), a cross-validation "inner" (mais interna- primeiro nível) seleciona o melhor modelo, enquanto a cross-validation "outer" (nível mais externo) nos proporciona uma avaliação do modelo não enviesada da performance. Na nossa solução, a cross-validation "inner"é o objeto do GridSearchCV, enquanto nos envelopamos a cross-validation "outer" usando cross_val_score.

Para tirar a confusão que esse processo traz. Façamos o seguinte experimento. Primeiramente, vamos setar verbose=1, no grid search para vermos o que está acontecendo, na procura pelo do melhor modelo através da seleção dos melhores parâmetros.

In [5]:
from sklearn.model_selection import (
    StratifiedKFold,
    KFold)

In [6]:
model = logistic
print("Os parâmetros do modelo são:")
for parameter in model.get_params():
    print(f"{parameter}")

Os parâmetros do modelo são:
C
class_weight
dual
fit_intercept
intercept_scaling
l1_ratio
max_iter
multi_class
n_jobs
penalty
random_state
solver
tol
verbose
warm_start


In [7]:
cv_inner = StratifiedKFold(
    n_splits = 5,
    shuffle=True,
    random_state=0)

model.set_params(max_iter=1000)
gridsearch = GridSearchCV(
    logistic,
    hyperparameters,
    cv = cv_inner,
    verbose = 1)

Vamos rodar o gridsearch, com a nossa "inner" cross-validation usada para achar o melhor modelo:

In [8]:
best_model = gridsearch.fit(features, target)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


Da saída da célula, vemos que a "inner" cross-validation treinou 20 candidatos, totalizando 100 modelos. Agora vamos colocar esse gridsearch dentro de outra validação cruzada.

In [9]:
%%time
cv_outer = KFold(
    n_splits = 5,
    shuffle=True, 
    random_state=0
    )

scores=cross_val_score(
    gridsearch, 
    features,
    target,
    cv=cv_outer,
    verbose=1, 
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
CPU times: user 41.4 s, sys: 143 ms, total: 41.6 s
Wall time: 41.5 s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   41.5s finished


A saída nos mostra que treinamos a inner cross-validation treinou 20 modelos 5 vezes para achar o melhor modelo e então esse modelo foi avaliado em uma "outer" cross-validation, criando um total de 500 modelos treinados.

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression())
])
pipeline

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', LogisticRegression())])

Queremos achar o melhor C através de um grid-search onde C possa tomar os seguintes valores C = [0.1, 1, 10]

In [11]:
pipeline.steps

[('scaler', StandardScaler()), ('classifier', LogisticRegression())]

In [13]:
print("Os parâmetros do pipeline são:")
for parameter in pipeline.get_params():
    print(parameter)

Os parâmetros do pipeline são:
memory
steps
verbose
scaler
classifier
scaler__copy
scaler__with_mean
scaler__with_std
classifier__C
classifier__class_weight
classifier__dual
classifier__fit_intercept
classifier__intercept_scaling
classifier__l1_ratio
classifier__max_iter
classifier__multi_class
classifier__n_jobs
classifier__penalty
classifier__random_state
classifier__solver
classifier__tol
classifier__verbose
classifier__warm_start


In [14]:
param_grid = {"classifier__C" : [0.1, 1, 10]}
model = GridSearchCV(
    pipeline,
    param_grid=param_grid
).fit(X, y)
model.best_params_

{'classifier__C': 10}

In [15]:
model.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', LogisticRegression(C=10))])

In [17]:
print("A melhor acurácia achada foi: ")
model.best_score_

A melhor acurácia achada foi: 


0.9733333333333334

In [18]:
model.cv_results_

{'mean_fit_time': array([0.01159291, 0.01415353, 0.0206151 ]),
 'std_fit_time': array([0.00280776, 0.00246526, 0.00227269]),
 'mean_score_time': array([0.00079789, 0.00063887, 0.00074949]),
 'std_score_time': array([1.59878287e-04, 6.14962077e-05, 2.00149499e-04]),
 'param_classifier__C': masked_array(data=[0.1, 1, 10],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'classifier__C': 0.1},
  {'classifier__C': 1},
  {'classifier__C': 10}],
 'split0_test_score': array([0.83333333, 0.96666667, 1.        ]),
 'split1_test_score': array([0.96666667, 1.        , 1.        ]),
 'split2_test_score': array([0.93333333, 0.93333333, 0.93333333]),
 'split3_test_score': array([0.9       , 0.9       , 0.93333333]),
 'split4_test_score': array([1., 1., 1.]),
 'mean_test_score': array([0.92666667, 0.96      , 0.97333333]),
 'std_test_score': array([0.05734884, 0.03887301, 0.03265986]),
 'rank_test_score': array([3, 2, 1], dtype=int32)}

* RandomizedSearchCV has a fixed computation budget through its n_iter parameter.
* GridSearchCV can become very computationally intensive when the number of parameters grows.
* both GridSearchCV and RandomizedSearchCV have the attributes cv_results_ and best_params_